# Ax Service-like API

When used as a service, Ax proposes new optimization trials, which contain parameterizations. The client application evaluates those parameterizations locally or using a scheduler and logs the results back to Ax, completing the trials. Then, Ax uses that data to propose more trials in the experiment.

In [1]:
from ax.service.ax_client import AxClient
from ax.metrics.branin import branin

The service-like API provides an ability to save an experiment to a database. To enable it, pass DB settings (`ax.storage.sqa_store.structs.DBSettings`) as the `db_settings` argument into `AxClient` constructor, and the experiment will be automatically saved any time there is a change. It can later be reloaded by experiment name. If instantiated without `DBSettings`, `AxClient` will not store the underlying experiment.

In [2]:
ax = AxClient()  # Or `AxClient(db_settings=my_db_settings)` to enable storage.

First step is to set up the experiment. It consists of a **search space** (parameters and parameter constraints) and **optimization configuration** (objective name, minimization setting, and outcome constraints). Note that:
- Only `name`, `parameters`, and `objective_name` arguments are required.
- Dictionaries in `parameters` have the following required keys: "name" - parameter name, "type" - parameter type ("range", "choice" or "fixed"), "bounds" for range parameters, "values" for choice parameters, and "value" for fixed parameters.
- Dictionaries in `parameters` can optionally include "value_type" ("int", "float", "bool" or "str"), "log_scale" flag for range parameters, and "is_ordered" flag for choice parameters.
- `parameter_constraints` should be a list of strings of form "p1 >= p2" or "p1 + p2 <= some_bound".
- `outcome_constraints` should be a list of strings of form "constrained_metric <= some_bound".

In [3]:
ax.create_experiment(  # Can use `ax.load_experiment(experiment_name) if reloading saved experiment.
    name="branin_test_experiment",
    parameters=[
        {
            "name": "x1",
            "type": "range",
            "bounds": [-5.0, 10.0],
            "value_type": "float",  # Optional, defaults to inference from type of "bounds".
            "log_scale": False,  # Optional, defaults to False.
        },
        {  
            "name": "x2",  # We'll omit the optional arguments for this parameter.
            "type": "range",
            "bounds": [0.0, 10.0],
        },
    ],
    objective_name="branin",
    minimize=True,  # Optional, defaults to False.
    parameter_constraints=["x1 + x2 <= 20"],  # Optional.
    outcome_constraints=["constrained_metric <= 10"],  # Optional.
)

When using Ax a service, evaluation of parameterizations suggested by Ax is done either locally or, more commonly, using an external scheduler. Below is a dummy evaluation function that outputs data for two metrics "branin" and "constrained_metric". Note that all returned metrics correspond to either the `objective_name` set on experiment creation or the metric names mentioned in `outcome_constraints`.

In [4]:
def evaluate(parameters):
    x1, x2 = parameters.get("x1"), parameters.get("x2")
    # For the sake of simplicity, we'll just use negative branin for constrained metric.
    return {"branin": (branin(x1, x2), 0.0), "constrained_metric": (-branin(x1, x2), 0.0)}

With the experiment set up, we can start the optimization loop. Note that Ax auto-selects an appropriate optimization algorithm based on the search space. For more advance use cases that require a specific optimization algorithm, pass a `generation_stratetegy` argument into the `AxClient` constructor.

In [5]:
for _ in range(15):
    parameters, trial_index = ax.get_next_trial()
     # Local evaluation here can be replaced with deployment to external system.
    ax.complete_trial(trial_index=trial_index, raw_data=evaluate(parameters))

Ax will use Bayesian optimization for this experiment, so optimization loop can take a few minutes.

Once it's complete, we can access the best parameters found, as well as the corresponding metric values.

In [6]:
best_parameters, metrics = ax.get_best_parameters()
best_parameters

{'x1': 9.286404856169058, 'x2': 1.0374315655961497}

In [7]:
means, covariances = metrics
means["branin"]

2.2408455525333864

# Special Cases

**Evaluation failure**: should any optimization iterations fail during evaluation, `log_trial_failure` will ensure that the same trial is not proposed again.

In [8]:
ax.log_trial_failure(trial_index=trial_index)

**Adding custom trials**: should there be need to evaluate a specific parameterization, `attach_trial` will add it to the experiment.

In [9]:
ax.attach_trial(parameters={"x1": 9.0, "x2": 9.0})

({'x1': 9.0, 'x2': 9.0}, 15)

**Need to obtain trials in batches**: use `AxBatchClient` instead of `AxClient`.